In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import yfinance as yf
from machine_learning_finance import analyze_trades, \
    calc_probabilties_without_lookahead, plot_backtest_analysis, \
    metrics_to_dataframe, create_train_test_windows, make_inverse_env_for
from plotly.offline import init_notebook_mode

init_notebook_mode(connected=True)
windows = [300, 600, 900, 1500]

symbol = "XLE"
inverse = "DUG"
file = f"../backtests/backtest_inverse_{symbol}_365_h0.9_l0.1.csv"
period = 365
pd.options.display.max_rows = None


def plot_win_loss(file, metrics):
    ledger = pd.read_csv(file)
    # Set the style for the plots
    sns.set(style='whitegrid')

    # Create a bar plot for profit and loss stats
    profit_stats, loss_stats = metrics['profit_stats'], metrics['loss_stats']
    profit_loss_df = pd.DataFrame([profit_stats, loss_stats], columns=['min', 'max', 'mean', 'median', 'std'], index=['profit', 'loss'])

    plt.figure(figsize=(12, 6))
    ax = sns.barplot(data=profit_loss_df.transpose(), palette='muted')
    ax.set_title('Profit and Loss Statistics')
    ax.set_ylabel('Value')

    # Show the plot
    plt.show()

def analyze_and_graph_range(file, symbol, start, end, df):
    ledger = pd.read_csv(file)
    hist_df, test_df = create_train_test_windows(df, start=start, end=end)
    test_df = calc_probabilties_without_lookahead(test_df, hist_df)
    plot_backtest_analysis(test_df, ledger, inverse=symbol)
    return ledger, test_df
   
def analyze_and_graph(file, symbol, period, df):
    ledger = pd.read_csv(file)
    hist_df = df.head(len(df)-period)
    hist_df = hist_df.tail(period*4)
    df = df.tail(period)
    df = calc_probabilties_without_lookahead(df, hist_df)
    plot_backtest_analysis(df, ledger, inverse=symbol)

    return ledger, df

ticker_obj = yf.download(tickers=symbol)
df = pd.DataFrame(ticker_obj)
#plot_win_loss(file, metrics)

#print(ledger)

#print(metrics_df)
#df.sort_index(ascending=False)

[*********************100%***********************]  1 of 1 completed


In [2]:
ledger, df2 = analyze_and_graph(file, symbol, 365*4, df)    
#ledger, df2 = analyze_and_graph_range(file, symbol, "03-27-2020", "12-17-2021", df)

# Display metrics as text
metrics = analyze_trades(ledger, symbol, period)
metrics["file"] = file
metrics_df = metrics_to_dataframe(metrics) 

merged_df = ledger.merge(df2, left_on='Date', right_index=True, how='left')
merged_df[["Date", "Price", "Product", "Side", "Action","prob_above_trend", "Value"]]

[*********************100%***********************]  1 of 1 completed


,Date,Price,Product,Side,Action,prob_above_trend,Value
0,2021-12-17,54.913699,XLE,long,enter,0.989756,4938.567001
1,2022-05-16,83.739104,XLE,long,exit,0.031575,7562.945132
2,2022-05-16,18.745599,DUG,long,enter,0.031575,7469.426571
3,2022-06-23,23.522901,DUG,long,exit,0.990546,9440.872442
4,2022-06-23,71.366604,XLE,long,enter,0.990546,9324.705656
5,2022-10-31,90.900000,XLE,long,exit,0.040002,11946.375426
6,2022-10-31,12.291700,DUG,long,enter,0.040002,11798.582187
7,2023-05-16,15.109600,DUG,long,exit,0.944987,14610.451544
8,2023-05-16,77.749803,XLE,long,enter,0.944987,14429.972809
9,2023-06-01,78.315401,XLE,long,exit,0.907174,14643.079704


In [3]:
df2.tail(35)


,Open,High,Low,Close,Adj Close,Volume,prob_above_trend,weighted-volume,trend,long_entry,long_exit,short_entry,short_exit
Date,,,,,,,,,,,,,
2023-04-24,84.709999,86.680000,84.680000,86.279999,86.279999,12625900,0.195482,1.089363e+09,83.124905,NaN,NaN,NaN,NaN
2023-04-25,85.440002,85.589996,84.160004,84.660004,84.660004,15332600,0.376591,1.298058e+09,83.071571,NaN,NaN,NaN,NaN
2023-04-26,84.059998,84.839996,83.019997,83.529999,83.529999,20035000,0.505356,1.673524e+09,83.018236,NaN,NaN,NaN,NaN
2023-04-27,83.480003,84.040001,82.690002,83.830002,83.830002,17057300,0.446855,1.429913e+09,82.964902,NaN,NaN,NaN,NaN
2023-04-28,83.529999,85.589996,83.239998,85.129997,85.129997,19110900,0.245904,1.626911e+09,82.911568,NaN,NaN,NaN,NaN
2023-05-01,84.059998,85.080002,83.720001,84.169998,84.169998,13910700,0.321333,1.170864e+09,82.858234,NaN,NaN,NaN,NaN
2023-05-02,83.160004,83.290001,79.739998,80.510002,80.510002,33824000,0.727525,2.723170e+09,82.804900,NaN,NaN,NaN,NaN
2023-05-03,79.480003,80.410004,78.959999,78.989998,78.989998,29683800,0.852248,2.344723e+09,82.751566,NaN,NaN,NaN,NaN
2023-05-04,79.019997,79.820000,77.870003,78.110001,78.110001,26430500,0.888431,2.064486e+09,82.698232,NaN,NaN,NaN,NaN


In [4]:
env = make_inverse_env_for(symbol,
                        inverse,
                        1,
                        period,
                        prob_high=0.9,
                        prob_low=0.1)
env.expert_opinion_df()
env.timeseries.tail(35)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Close,weighted-volume,trend,prob_above_trend,action
Date,,,,,
2023-04-24,0.798554,0.056775,0.696426,0.287039,0
2023-04-25,0.756715,0.087004,0.693274,0.387251,0
2023-04-26,0.727531,0.141391,0.690122,0.438436,0
2023-04-27,0.735279,0.106104,0.686971,0.445907,0
2023-04-28,0.768853,0.134639,0.683819,0.334502,0
2023-05-01,0.744060,0.068580,0.680667,0.426497,0
2023-05-02,0.649535,0.293434,0.677515,0.670805,0
2023-05-03,0.610279,0.238616,0.674363,0.724256,0
2023-05-04,0.587552,0.198023,0.671211,0.733765,0
